In [2]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Adding filter query (fq) for more precise control
    fq = 'fq=body:("Old Navy") AND body:("clothing") AND NOT body:("Trump") AND NOT body:("Israel")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("Old Navy", api_key, page = i)
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)


    
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])
text

def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('Files_Directory/Twitter/AFINN-111.txt')
# Scores_Table = read_scores('AFINN-111.txt')

scores = []

for txt in text:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)
!pip install azure-ai-textanalytics

import requests
import azure
import time

key = 'ce14ce8f012d462fbbedd4c70565b678' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://fischba-webdataproject-fall2024.openai.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
        
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)
    
final_sentiments

print(final_sentiments.count('positive'))
print(final_sentiments.count('negative'))

0
1
2
3
4
5
6
7
8
9
1
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), 'sentences': [SentenceSentiment(text=Her Job Requires 7 Apps. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.0), length=25, offset=0, mined_opinions=[]), SentenceSentiment(text=She Works Retail.. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), length=19, offset=25, mined_opinions=[]), SentenceSentiment(text=Responding to the customer’s demand for instant gratification in the middle of the retail apocalypse., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.09, neutral=0.84, negative=0.07), length=101, offset=44, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
2
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidenc

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.01), 'sentences': [SentenceSentiment(text=Kanye West and Gap Strike 10-Year Deal for ‘Yeezy Gap’ Apparel Line. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.96, negative=0.02), length=69, offset=0, mined_opinions=[]), SentenceSentiment(text=The struggling mall brand plans to introduce Yeezy Gap in stores and online in the first half of 2021., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.01), length=102, offset=69, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
10
{'id': '0', 'sentiment': 'positive', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.53, neutral=0.46, negative=0.0), 'sentences': [SentenceSentiment(text=Gap, Looking for Barbie Magic, Hires Mattel’s Pre

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.16, negative=0.84), 'sentences': [SentenceSentiment(text=Pandemic Dressing Takes a Dark Turn. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.96, negative=0.04), length=37, offset=0, mined_opinions=[]), SentenceSentiment(text=As we start another year, still in isolation, our clothing choices are getting increasingly weird., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.16, negative=0.84), length=98, offset=37, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
20
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.04, neutral=0.96, negative=0.0), 'sentences': [SentenceSentiment(text=How Data Will Change The Future of Customer Service. , sentiment=neutral, confidence_sc

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.86, negative=0.12), 'sentences': [SentenceSentiment(text=After a Bumper 2021, Companies Might Struggle to Increase Profits. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), length=67, offset=0, mined_opinions=[]), SentenceSentiment(text=Businesses face headwinds as demand weakens, the Federal Reserve raises rates and government stimulus programs end., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.78, negative=0.21), length=115, offset=67, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
31
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.7, negative=0.29), 'sentences': [SentenceSentiment(text=Westfield Gives Up San Francisco Mall,

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.92, negative=0.06), 'sentences': [SentenceSentiment(text=Chaos and Creation: Inside the Making of Yeezy Gap. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.92, negative=0.06), length=52, offset=0, mined_opinions=[]), SentenceSentiment(text=In 2020, two fashion brands announced an unusual alliance. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), length=59, offset=52, mined_opinions=[]), SentenceSentiment(text=Now that goods are finally hitting stores, is Yeezy Gap a corporate-creative cautionary tale, or a new model for fashion to come?, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.04, neutral=0.85, negative=0.12), length=129, offset=111, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
40
{'id'

In [7]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Filter to search for "Old Navy" in the headline and specific section
    fq = 'fq=headline:("Old Navy") AND section_name:("Fashion" OR "Business")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("Old Navy", api_key, page = i)
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)


    
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])
text

def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('Files_Directory/Twitter/AFINN-111.txt')
# Scores_Table = read_scores('AFINN-111.txt')

scores = []

for txt in text:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)
!pip install azure-ai-textanalytics

import requests
import azure
import time

key = 'ce14ce8f012d462fbbedd4c70565b678' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://fischba-webdataproject-fall2024.openai.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
        
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)
    
final_sentiments

print(final_sentiments.count('positive'))
print(final_sentiments.count('negative'))

0


KeyboardInterrupt: 

In [10]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Filter to search for "Old Navy" in the headline and specific section
    fq = 'fq=body:("Old Navy") AND section_name:("Fashion" OR "Business")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()



IndexError: list index out of range